In [100]:
import csv
import cv2
import numpy as np

In [101]:
#read csv data file
lines = []

with open('./data/driving_log.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    for row in csvReader:
        lines.append(row)

In [102]:
lines[0]

['/Users/manika/Desktop/data/IMG/center_2017_10_25_18_26_51_303.jpg',
 '/Users/manika/Desktop/data/IMG/left_2017_10_25_18_26_51_303.jpg',
 '/Users/manika/Desktop/data/IMG/right_2017_10_25_18_26_51_303.jpg',
 '0',
 '0',
 '0',
 '9.12E-07']

In [114]:
#read the center images
images = []
measurements = []
for line in lines:
#     for i in range(3):
#         imagePath = line[i]
    sourcePath = line[i]
    fileName = sourcePath.split('/')[-1]
    currentPath = './data/IMG/'+fileName
    image = cv2.imread(currentPath)
#         image = cv2.imread(imagePath)
    images.append(image)
    measurement = float (line[3])
    measurements.append(measurement)

In [120]:
len(images)

2388

In [121]:
print (images[0])

[[[185 145 116]
  [185 145 116]
  [184 144 115]
  ..., 
  [ 24  36  30]
  [ 37  52  44]
  [  7  22  14]]

 [[185 145 116]
  [185 145 116]
  [185 145 116]
  ..., 
  [  0   3   0]
  [  0   6   0]
  [  0  13   7]]

 [[186 146 117]
  [186 146 117]
  [186 146 117]
  ..., 
  [ 16  29  27]
  [ 20  33  31]
  [ 26  41  37]]

 ..., 
 [[173 187 175]
  [157 171 159]
  [142 154 142]
  ..., 
  [221 236 255]
  [228 242 255]
  [220 233 255]]

 [[169 183 171]
  [152 166 154]
  [138 151 137]
  ..., 
  [241 251 255]
  [202 210 233]
  [241 249 255]]

 [[166 180 168]
  [149 163 151]
  [136 149 135]
  ..., 
  [245 254 255]
  [248 254 255]
  [241 246 255]]]


In [122]:
measurements[0]

0.0

In [123]:
#data augmentation
augmentedImages, augmentedMeasurements = [], []
for image, measurement in zip(images, measurements):
    augmentedImages.append(image)
    augmentedMeasurements.append(measurement)
    augmentedImages.append(cv2.flip(image, 1))
    augmentedMeasurements.append(measurement*-1.0)

In [124]:
#convert the images and measurements to train data and label
# X_train = np.array(images)
# y_train = np.array(measurements)
X_train = np.array(augmentedImages)
y_train = np.array(augmentedMeasurements)

In [125]:
len(X_train)

4776

In [ ]:
#build a basic neural network to check if everything is working
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Activation
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D

model = Sequential()
model.add(Lambda(lambda x:x/255.0-0.5, input_shape=(160, 320, 3)))
model.add(Convolution2D(6, (5, 5)))
model.add(Activation("relu"))
model.add(MaxPooling2D())
model.add(Convolution2D(6, (5, 5)))
model.add(Activation("relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split = 0.2, shuffle=True, epochs=4) #nb_epoch is set to 7 as after this the loss increses

model.save('modelAug.h5')

Train on 3820 samples, validate on 956 samples
Epoch 1/4
 320/3820 [=>............................] - ETA: 139s - loss: 90.8542 